# CNN-MNIST

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
np.random.seed(20181105)
tf.set_random_seed(20181105)

**load data**

In [7]:
mnist = input_data.read_data_sets('./DataSet/tmp/data', one_hot=True)

Extracting ./DataSet/tmp/data\train-images-idx3-ubyte.gz
Extracting ./DataSet/tmp/data\train-labels-idx1-ubyte.gz
Extracting ./DataSet/tmp/data\t10k-images-idx3-ubyte.gz
Extracting ./DataSet/tmp/data\t10k-labels-idx1-ubyte.gz


In [8]:
num_filter1 = 32 #필터의 개수
x = tf.placeholder(tf.float32, [None, 784]) #28*28*1

#연산의 편의를 위해 2차원 행렬 할 것임. 784(1차원 벡터) -> 28*28*1
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1 = None(28*28*1 행렬을 무한개[-1])

# W_conv1의 차원(shape) : [5,5,1,32]
W_conv1 = tf.Variable(tf.random_normal([5,5,1,num_filter1]))

**필터에 입력 이미지 적용**

* strides는 통상적으로 맨 앞뒤에는 1을 씀. 
중간에 있는 두 값이 실질적인 가로/세로 방향 이동 칸 수

* 패딩 : 필터를 적용하면 추출된 특징 행렬은 원 이미지보다 크기가 작아진다.
필터 계속 적용 -> 특징이 유실될 문제가 발생될 수 있음 -> 패딩 적용

* 풀링(pooling) : max pooling을 적용\
ksize = 풀링시 필터(커널)의 크기-2*2 크기로 묶어서 풀링하겠다


In [9]:
# strides / 패딩
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding = "SAME")
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filter1]))

#  활성화 함수 적용(CNN-Relu함수) 
h_conv1_cutoff = tf.nn.relu(h_conv1+b_conv1)

# 풀링
h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides = [1,2,2,1], padding = "SAME")

# 행렬의 차원 변환
num_filter2 = 64 #필터의 개수
W_conv2 = tf.Variable(tf.random_normal([5,5,num_filter1,num_filter2])) 
#[5,5,32,64] , 필터 크기 : 5*5, 입력되는 값: 32개, 32개가 들어가서 총 64개의 필터가 적용됨
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding = "SAME")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filter2]))
h_conv2_cutoff = tf.nn.relu(h_conv2+b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides = [1,2,2,1], padding = "SAME")


* 28*28 -> 첫번째 풀링(2*5) ->14*14->두번째 풀링(2*2)->7*7
* 결국 7*7 크기의 행렬이 64개가 나오게 됨

* 풀리 커넥티드 계층
* 64개의 입력으로부터 ==> 10개의 숫자로 분류
* 두번째 컨볼루셔널 계층에서 특징을 뽑아냈으면, 이 특징으로 0~9까지의 숫자를 판별하기 위한 fully connected layer 구현

In [10]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filter2]) # 입력된 64개의 7*7 행렬을 1차원 행렬로 변환
num_units1 = 7*7*num_filter2
num_ubits2 = 1024
w2 = tf.Variable(tf.random_normal([num_units1, num_ubits2]))
b2 = tf.Variable(tf.constant(0.1, shape = [num_ubits2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) +b2)
keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

**모델 정의**

In [11]:
w0 = tf.Variable(tf.zeros([num_ubits2, 10]))
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(hidden2_drop, w0)+b0
p = tf.nn.softmax(k)

**비용함수 정의**

In [12]:
# cost : 크로스엔트로피 함수 적용
t = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=k, labels=t))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

**학습수행**

In [15]:
i = 0
for _ in range(10000) :
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []  # 학습비용, 정확도
        for c in range(4): #test, labels가 1만개
            start = int(len(mnist.test.labels)/4*c) #0
            end = int(len(mnist.test.labels)/4*(c+1)) #2500
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[start:end], t:mnist.test.labels[start:end], keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            
            loss_val = np.sum(loss_vals)
            acc_val = np.mean(acc_vals)
            
            print("step : %d, Loss : %f, Accuracy : %f" % (i, loss_val, acc_val))

step : 500, Loss : 2.160685, Accuracy : 0.953600
step : 500, Loss : 4.095010, Accuracy : 0.956200
step : 500, Loss : 5.211536, Accuracy : 0.965067
step : 500, Loss : 5.940545, Accuracy : 0.968200
step : 1000, Loss : 2.228215, Accuracy : 0.952400
step : 1000, Loss : 4.340529, Accuracy : 0.954800
step : 1000, Loss : 5.394093, Accuracy : 0.964133
step : 1000, Loss : 6.096089, Accuracy : 0.968400
step : 1500, Loss : 2.167044, Accuracy : 0.956400
step : 1500, Loss : 4.072395, Accuracy : 0.956600
step : 1500, Loss : 5.008822, Accuracy : 0.965067
step : 1500, Loss : 5.850277, Accuracy : 0.968200
step : 2000, Loss : 2.024672, Accuracy : 0.956400
step : 2000, Loss : 3.827759, Accuracy : 0.958200
step : 2000, Loss : 4.766872, Accuracy : 0.966667
step : 2000, Loss : 5.415824, Accuracy : 0.969900
step : 2500, Loss : 2.073530, Accuracy : 0.956000
step : 2500, Loss : 3.802145, Accuracy : 0.957800
step : 2500, Loss : 4.741195, Accuracy : 0.965333
step : 2500, Loss : 5.667326, Accuracy : 0.968200
step